In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests 
import time
import re

# pip install geopy
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from geopy_functions import *
from my_functions import *

## Coses pendents, artistes a revisar

Hi ha dos grups que es diuen 'Embrace', un de UK i un de USA

**YOB** te **8** albums a Sputnik, però nomes **2** a Discogs de UK, tot i que és una banda de UK. 

Si apareixen al df de US, hauré de fer un concat/merge

In [141]:
df[df['artist']=='YOB']

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
7104,2003,YOB,Catharsis,3.99,140,49.07,3,['Doom Metal'],US,Yob is an American stoner/doom metal band from...
9049,2014,YOB,Clearing The Path To Ascend,3.92,331,62.53,4,"['Doom Metal', 'Sludge Metal']",US,Yob is an American stoner/doom metal band from...


In [142]:
df_ratings_20[df_ratings_20['artist']=='YOB']

,album_id,artist,title,rating,votes
2485,3646,YOB,The Illusion Of Motion,3.94,183
15507,29146,YOB,The Unreal Never Lived,4.06,225
19335,39606,YOB,The Great Cessation,3.90,205
24401,60240,YOB,Elaborations Of Carbon,3.42,73
24402,60241,YOB,Catharsis,3.99,140
28092,80046,YOB,Atma,3.78,259
36277,163668,YOB,Clearing The Path To Ascend,3.92,331
43372,282109,YOB,Our Raw Heart,3.87,295


In [148]:
df_uk_masters[df_uk_masters['artist'].str.contains('^YOB', case=False)]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
8995,563922.0,729329.0,6054020,US,Yob,Clearing The Path To Ascend,2014,62.53,4,['Album'],['Rock'],"['Doom Metal', 'Sludge Metal']",Yob is an American stoner/doom metal band from...
38287,14529679.0,3488023.0,30569908,UK,Yobs (4),Yobs,2024,0.00,10,"['LP', '45 RPM', 'Album', 'Limited Edition', '...",['Rock'],"['Punk', 'Garage Rock']",Liverpool Garage Punks...
42278,563922.0,129480.0,2806549,US,Yob,Catharsis,2003,49.07,3,['Album'],['Rock'],['Doom Metal'],Yob is an American stoner/doom metal band from...


In [149]:
df_us_masters[df_us_masters['artist'].str.contains('^YOB', case=False)]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


# **Locations Wikipedia scraping**

## Import dataframes

In [2]:
df_ratings_10 = pd.read_csv('Datasets/df_ratings_10.csv', keep_default_na=False)

print(f"{df_ratings_10['artist'].nunique()} artists")
print(f"{df_ratings_10.shape[0]} albums")
print(f"Average of {round(df_ratings_10.shape[0] / df_ratings_10['artist'].nunique(), 2)} albums per artist in the df that has all the albums with any ratings")

23683 artists
79625 albums
Average of 3.36 albums per artist in the df that has all the albums with any ratings


In [25]:
df = pd.read_csv('Datasets/df_rock_ratings.csv', keep_default_na=False)

print(f"{df['artist'].nunique()} artists")
print(f"{df.shape[0]} albums")
print(f"Average of {round(df.shape[0] / df['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 10 votes")

5095 artists
12059 albums
Average of 2.37 albums per artist in the subset with the (mostly UK) albums with more than 10 votes


In [9]:
df_20 = pd.read_csv('Datasets/df_rock_ratings_20.csv', keep_default_na=False)

print(f"{df_20['artist'].nunique()} artists")
print(f"{df_20.shape[0]} albums")
print(f"Average of {round(df_20.shape[0] / df_20['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 20 votes")

3823 artists
9380 albums
Average of 2.45 albums per artist in the subset with the (mostly UK) albums with more than 20 votes


In [ ]:
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv')
df_ratings_20.shape

(51252, 5)

## Testing

In [28]:
df.sample(5)

,artist,title,year,album_length,tracks,genres,styles,release_country,artist_profile
10580,The 1975,A Brief Inquiry into Online Relationships,2018,58.43,15,"['Rock', 'Pop']","['Indie Rock', 'Alternative Rock', 'Indie Pop']",UK & Europe,British indie rock band. \r\n\r\nPop-rock band...
9889,Le Butcherettes,A Raw Youth,2015,0.00,12,['Rock'],[],US,Formed by Teri Gender Bender and Auryn Jolene ...
6394,John Fogerty,Centerfield,1985,35.33,9,['Rock'],"['Pop Rock', 'Folk Rock', 'Country Rock']",US,"American musician, songwriter, and guitarist (..."
4498,L7,The Beauty Process: Triple Platinum,1997,41.57,12,['Rock'],"['Punk', 'Grunge']",US,American grunge punk/alternative rock band fro...
5742,The Fall,Are You Are Missing Winner,2001,47.68,10,['Rock'],"['Garage Rock', 'Punk', 'Rockabilly']",UK,"Post-punk band from Greater Manchester, UK. 19..."


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [658]:
artist = "John Grant"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
name_changed_musician = name_changed + ('_(musician)')

url = f"https://en.wikipedia.org/wiki/{name_changed_musician}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the musician is dead
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Genres" in text:
                        location = text.split('Genres')[0]
                        print('Genres')
                    else:
                        if "Occupations" in text:
                            location = text.split('Occupations')[0]
                            print('Occupations')
                        else:
                            location = np.nan
        except:  
            location = np.nan
except:
    print('fuck')


Genres


In [659]:
location

'Buchanan, Michigan, U.S.'

In [652]:
name_changed_musician

'John_Grant_(musician)'

In [533]:
artist = "Stone Gossard"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')

url = f"https://en.wikipedia.org/wiki/{name_changed}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    location = table[0].text.split('Origin')[1].split('Genres')[0]

# save info in lists
    print('origin')

except:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the artist is dead
        print('dead')
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Occupations" in text:
                        location = text.split('Occupations')[0]
                    else:
                        if "Genres" in text:
                            location = text.split('Genres')[0]
                            print(repr(location))
                        else:
                            location = np.nan
        except:  
            location = np.nan


In [535]:
text

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metalOccupationsMusiciansongwriterInstrumentsGuitarvocalsYears active'

In [536]:
location

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metal'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## **Changing dataframes**

### **``df_artists_origins``**

In [93]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins.shape

(3251, 2)

In [94]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [95]:
df_artists_origins[df_artists_origins['origin']=='United Kingdom']

,artist,origin


In [86]:
df_new_artists = df_artists_origins[df_artists_origins['origin']=='United Kingdom']

In [89]:
df_new_artists

,artist,origin
1059,Son of Dork,"London, England"
1192,Mojave 3,"London, England"
1326,Crippled Black Phoenix,"Bristol, England"
1597,This Mortal Coil,"Wandsworth, England"
2191,Arcadia,"London, England"
2300,Jade Warrior,"London, England"
2348,The Waterboys,"London, England"
2469,Blackmore's Night,"Mount Sinai, NY, United States"
2556,Atomic Rooster,"London, England"
2609,The Nefilim,"Lambeth, England"


In [88]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Son of Dork", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Mojave 3", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Crippled Black Phoenix", "Bristol, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="This Mortal Coil", "Wandsworth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Arcadia", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jade Warrior", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Waterboys", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Blackmore's Night", "Mount Sinai, NY, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Atomic Rooster", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Nefilim", "Lambeth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Spiders", "Sheffield, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brontide", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gilgamesh", "Hampstead, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Young Legionnaire", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Deviants", "Ladbroke Grove, London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Head of David", "Dudley, West Midlands, United Kingdom", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Quintessence", "London, England", df_new_artists["origin"])

In [53]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[8016]['artist_profile']
splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Formed in 1969, they played a blend of jazz, progressive rock, Indian Music, and new age rock.   Members included:
Sambhu Babaji : Bass  Dave Codling : Guitar  Shiva Shankar Jones : Keyboards, Vocals  Jake Milton : Drums  Alan Mostert
: Guitar  Raja Ram : Flute, Piano, Vocals


In [52]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Quintessence".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
8016,1969,Quintessence,In Blissful Company,3.88,16,0.0,8,['Psychedelic Rock'],UK,"Formed in 1969, they played a blend of jazz, p..."


In [90]:
df_artists_origins = df_artists_origins[df_artists_origins['origin']!='United Kingdom']
df_artists_origins.shape

(3234, 2)

In [91]:
df_artists_origins.to_csv('Datasets/df_artists_origins.csv', index=False)

In [92]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3251, 2)


In [56]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/9 - London, Greater London, England, United Kingdom
2/9 - Bristol, City of Bristol, West of England, England, United Kingdom
3/9 - Wandsworth, London Borough of Wandsworth, London, Greater London, England, SW18 1UJ, United Kingdom
4/9 - Mount Sinai, Miller Place, Town of Brookhaven, Suffolk County, New York, 11766, United States
5/9 - Lambeth, London Borough of Lambeth, London, Greater London, England, SE1 7JW, United Kingdom
6/9 - Sheffield, South Yorkshire, England, United Kingdom
7/9 - Hampstead, Greater London, England, NW3 1QG, United Kingdom
8/9 - Ladbroke Grove, Westway, Lancaster West Estate, North Kensington, Royal Borough of Kensington and Chelsea, London, Greater London, England, W10 5YG, United Kingdom
9/9 - Dudley, West Midlands, England, United Kingdom


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Bristol,"Bristol, England","Bristol, England",51.453802,-2.597298,"Bristol, City of Bristol, West of England, Eng..."
1,United Kingdom,Dudley,"Dudley, West Midlands, United Kingdom","Dudley, West Midlands, United Kingdom",52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
2,United Kingdom,Hampstead,"Hampstead, England","Hampstead, England",51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3,United Kingdom,Ladbroke Grove,"Ladbroke Grove, London, England","Ladbroke Grove, London, England",51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
4,United Kingdom,Lambeth,"Lambeth, England","Lambeth, England",51.495211,-0.116335,"Lambeth, London Borough of Lambeth, London, Gr..."
5,United Kingdom,London,"London, England","London, England",51.507446,-0.127765,"London, Greater London, England, United Kingdom"
6,United Kingdom,Sheffield,"Sheffield, England","Sheffield, England",53.380663,-1.470228,"Sheffield, South Yorkshire, England, United Ki..."
7,United Kingdom,Wandsworth,"Wandsworth, England","Wandsworth, England",51.457027,-0.193261,"Wandsworth, London Borough of Wandsworth, Lond..."
8,United States,Mount Sinai,"Mount Sinai, NY, United States","Mount Sinai, NY, United States",40.941066,-73.019455,"Mount Sinai, Miller Place, Town of Brookhaven,..."


### **``df_coordinates``**

In [57]:
df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scraped.shape

(1527, 7)

In [58]:
df_coordinates_scraped.head()

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Adelaide,"Adelaide, South Australia, Australia","Adelaide, South Australia, Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
1,Australia,Adelaide,"Adelaide, South Australia","Adelaide, South Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
2,Australia,Blue Mountains,"Blue Mountains, NSW, Australia","Blue Mountains, NSW, Australia",-33.609741,150.405224,"Blue Mountains, New South Wales, Australia"
3,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
4,Australia,Byron Bay,"Byron Bay, New South Wales, Australia","Byron Bay, New South Wales, Australia",-28.648333,153.617778,"Byron Bay, Byron Shire Council, New South Wale..."


In [59]:
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address
702,United Kingdom,United Kingdom,United Kingdom,United Kingdom,54.702354,-3.276575,United Kingdom


In [60]:
df_coordinates_scraped.drop(702, axis=0, inplace=True)
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address


In [62]:
df_coordinates_scraped.to_csv('Datasets/df_coordinates.csv', index=False)

In [63]:
df_coordinates.shape

(9, 7)

In [64]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1526, 7)

Found 4 duplicates:
               city         country
324         Bristol  United Kingdom
515  Ladbroke Grove  United Kingdom
542          London  United Kingdom
650       Sheffield  United Kingdom

Resulting dataset: (1531, 7)
Merged artists with coordinates! Found 5 new locations
df_coordinates_concat exported to .csv


### **``df_artists_origins_coordinates_concat``**

In [74]:
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3246,Gilgamesh,United Kingdom,Hampstead,51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3247,Young Legionnaire,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
3248,The Deviants,United Kingdom,Ladbroke Grove,51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
3249,Head of David,United Kingdom,Dudley,52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
3250,Quintessence,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


## **Changing values**

In [49]:
df

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
0,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,"['Indie Rock', 'Alternative Rock']",US,"El Paso, Texas based band. Formed in 2001 afte..."
1,2003,Muse,Absolution,3.99,4411,52.18,14,['Alternative Rock'],Europe,"Muse are an English rock band from Teignmouth,..."
2,1999,Muse,Showbiz,3.50,2181,49.65,12,"['Alternative Rock', 'Prog Rock']",UK,"Muse are an English rock band from Teignmouth,..."
3,2002,Finch,What It Is to Burn,3.69,864,61.18,13,"['Emo', 'Post-Hardcore', 'Alternative Rock']",US,"Finch is a post-hardcore band from Temecula, C..."
4,2002,Transplants,Transplants,3.35,198,45.23,12,['Punk'],US,"Punk rock/rap rock band Los Angeles, CA, Unite..."
...,...,...,...,...,...,...,...,...,...,...
12054,2024,As Living Arrows,Hope and Ruin,3.80,47,0.00,8,['Post-Hardcore'],NaN,"Post-screamo band from Brighton, UK\r\n"
12055,2024,The Dopamines,80/20,3.92,13,0.00,13,"['Punk', 'Pop Punk']",US,"Punk band from Cincinnati, Ohio."
12056,2024,Dawnwalker,The Unknowing,3.68,11,0.00,9,['Prog Rock'],UK,"Post-Metal/Rock band from London, UK."
12057,2024,Tiny Moving Parts,Deep in the Blue,3.02,28,0.00,10,"['Emo', 'Math Rock', 'Pop Punk']",US,"Trio from Benson, Minnesota. The band takes in..."


In [579]:
df[df['artist']=='Nucleus']

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile


In [580]:
df[df['artist'].str.contains('Nucleus')]

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
7537,1970,Nucleus (UK),Elastic Rock,3.55,20,0.00,13,"['Jazz-Rock', 'Fusion', 'Prog Rock']",UK,"Pioneering jazz-rock, progressive, psychedelic..."
7568,1971,Nucleus (UK),We'll Talk About It Later,3.79,21,0.00,7,"['Fusion', 'Jazz-Funk', 'Jazz-Rock', 'Prog Rock']",UK,"Pioneering jazz-rock, progressive, psychedelic..."
9833,2016,Nucleus (US),Sentient,3.29,29,37.92,9,['Death Metal'],US,"Death Metal band from Chicago, Illinois, USA. ..."
10933,2019,Nucleus (US),Entity,3.68,40,38.42,8,['Death Metal'],US,"Death Metal band from Chicago, Illinois, USA. ..."


In [578]:
df.loc[7537, 'artist'] = 'Nucleus (UK)'
df.loc[7568, 'artist'] = 'Nucleus (UK)'
df.loc[9833, 'artist'] = 'Nucleus (US)'
df.loc[10933, 'artist'] = 'Nucleus (US)'

In [869]:
df.drop([7660, 8037], axis=0, inplace=True)
df.shape

(12059, 10)

In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [582]:
df.to_csv('Datasets/df_rock_ratings.csv', index=False)

In [581]:
df.shape

(12059, 10)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## Defining the functions

In [15]:
def get_origins_wikipedia(df, start_index, final_index):
    df = pd.read_csv('Datasets/df_rock_ratings_20.csv')
    artists = df['artist'].unique()

    try:
    # import the DataFrame with the locations whose coordinates I already have
        df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
        print('Bingo! df_coordinates.csv found \n')
    except: 
        print('df_coordinates.csv not found \n')

    artists_list = []
    origin_list = []
    count=0
    scraped=0

    for index in artists_us_to_do[start_index:final_index]:

        name_changed = index.replace(' ', '_')
        name_changed_band = name_changed + ('_(band)')

        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            location = table[0].text.split('Origin')[1].split('Genres')[0]
            city = location.split(', ')[0]
            count+=1
            
        # save info in lists
            artists_list.append(index)  
            origin_list.append(location)
            scraped+=1
            print(f'{scraped}/{count} - {name_changed_band}: {location}')

        except:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")
                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                try:
                    location = table[0].text.split('Origin')[1].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1 
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed}: {location}')

                except:
                    location = table[0].text.split(')')[2].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1

                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (individual): {location}')

            except:
                try:
                    url = f"https://es.wikipedia.org/wiki/{name_changed}"
                    response = requests.get(url).content
                    soup = BeautifulSoup(response, "html.parser")

                    table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')
                    location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                    city = location.split(', ')[0]
                    count+=1    
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (español): {location}')

                except:
                    count+=1
                    print(f'{scraped}/{count} - {index}: error')
                    artists_list.append(index) 
                    origin_list.append(np.nan)

        if len(artists_list) != len(origin_list):
            print('different lengths')
            break

    df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
    
    return df_artists_origins

In [102]:
def get_new_artists(df_artists_origins):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

    if df_artists_origins['origin'].isna().sum() == 0:        
        print("No null values, but let's take a look just in case there are weird locations")

    else: 
    # take a look at the df with the new artists and make sure there are non null values in origin (when it couldn't find it in Wikipedia)
        print(f'{round(df_artists_origins['origin'].isna().sum() / df_artists_origins.shape[0]*100, 2)} % of nulls')
    
# subset of the new artists I just got, wether there are null values or not
    df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]

    print("Here is the dataframe with the new artists, without duplicates")
    return df_new_artists   # so I can take a look at it and then continue

In [ ]:
def export_artists_origins_concat(df_new_artists):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

# concat with the df I just got
    df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
    df_artists_origins_concat.drop_duplicates(inplace=True)     # just in case
    df_artists_origins_concat.reset_index(drop=True, inplace=True)

# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
    df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)
    print('df_artists_origins_concat exported to .csv')
    print(df_artists_origins_concat.shape)

In [175]:
def get_coordinates_geopy(df_new_artists):
    
# replace special characters for spaces
    df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
    df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# run the function that gets the coordinates from the origins from Geopy
    geolocator = Nominatim(user_agent="music_analysis", timeout=10)

# if they are 'dirty' origins that after the cleaning, they result in the same 'origin_clean'
    df_unique = df_new_artists[['origin', 'origin_clean']].drop_duplicates() 
    unique_origins = df_unique['origin'].values
    unique_origins_clean = df_unique['origin_clean'].values

    country_list = []
    city_list = []
    latitude_list = []
    longitude_list = []
    address_list = []
    lists = [country_list, city_list, latitude_list, longitude_list, address_list]
    count = 0

    for origin in unique_origins_clean:
        count+=1
        time.sleep(1)
        location = geolocator.geocode(origin)

        print(f'{count}/{len(unique_origins_clean)} - {location.address}')  

    # save the info in lists
        country_list.append(location.address.split(', ')[-1])
        city_list.append(origin.split(', ')[0])
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
        address_list.append(location.address)

        # # Check lengths
        # print(f"{count}/{len(unique_origins_clean)} - {origin}")
        # print(f"Current list lengths -> country: {len(country_list)}, city: {len(city_list)}, "
        #     f"lat: {len(latitude_list)}, lon: {len(longitude_list)}, address: {len(address_list)}")

    df_coordinates = pd.DataFrame({'country': country_list
                                , 'city': city_list
                                , 'origin': unique_origins
                                , 'origin_clean': unique_origins_clean
                                , 'latitude': latitude_list
                                , 'longitude': longitude_list
                                , 'address': address_list})
    df_coordinates.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates.reset_index(drop=True, inplace=True)

    return df_coordinates

In [ ]:
def export_coordinates_concat(df_coordinates):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
    print(f"df_coordinates_scraped: {df_coordinates_scraped.shape}\n")

# concat with the df of the coordinates I just got
    df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
    df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates_concat.reset_index(drop=True, inplace=True)

# look for duplicates in the origin, between the locations I had already gotten and the new ones
    check_duplicates_origins(df_coordinates_concat)
    new_origins = df_coordinates_concat.shape[0] - df_coordinates_scraped.shape[0]
    print(f"Merged artists with coordinates! Found {new_origins} new locations")

# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
    df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)
    print('df_coordinates_concat exported to .csv')

In [140]:
def merge_origins_coordinates(df_new_artists):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
    df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
    df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

# import the df that contains info of the artists and the coordinates of their origins
    df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# concat to get the df with all the artists, origins and their coordinates
    df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
    df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)

# save it in a csv file
    df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)
    print("Exported to a .csv file")

    return df_artists_origins_coordinates_concat

## Merge dataframes and look for the ``new_artists``

In [4]:
df_blend_ratings = pd.read_csv('Datasets/df_blend_ratings.csv')
artists_blend = df_blend_ratings['artist'].unique()
len(artists_blend)

4249

In [5]:
df = pd.read_csv('Datasets/df_rock_ratings_20.csv')
artists = df['artist'].unique()
artists_us = []

for artist in artists_blend:
    if artist not in artists:
        artists_us.append(artist)

len(artists_us)

459

In [47]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists_us_to_do = []

for artist in artists_us:
    if artist not in df_artists_origins['artist'].values:
        artists_us_to_do.append(artist)

len(artists_us_to_do)

357

In [7]:
df_artists_origins

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."
...,...,...
3419,Cinderella,"Philadelphia, Pennsylvania, U.S."
3420,Death From Above 1979,"Toronto, Ontario, Canada"
3421,Chicago,"Chicago, Illinois, United States"
3422,The Nation of Ulysses,"Washington, D.C."


In [8]:
artists_us_to_do

['The Band',
 'Hootie and The Blowfish',
 'John Frusciante',
 'Necros',
 'Minus the Bear',
 'Gorilla Biscuits',
 'Sausage',
 'Between the Buried and Me',
 'Bon Jovi',
 'Hypocrisy',
 'Rory Gallagher',
 'Slipknot',
 'Bob Dylan and The Band',
 'Protest the Hero',
 'Lagwagon',
 'Jason Becker',
 'The Bronx',
 'Chris Bell',
 'Jackyl',
 'Insomnium',
 'New York Dolls',
 'Vio-Lence',
 'Ulver',
 'Down By Law',
 'Tankard',
 'Kid Dynamite',
 'Forced Entry',
 'Sacrifice',
 'Rites of Spring',
 'Kid Rock',
 'I Mother Earth',
 'The Mighty Mighty Bosstones',
 'Quiet Riot',
 'Embrace',
 'Uncle Slam',
 'Red House Painters',
 'Angry Samoans',
 'Vince Neil',
 'Strawberry Alarm Clock',
 'Enslaved',
 'Beherit',
 'Guttermouth',
 'Acid Bath',
 'ALL',
 'Caribou',
 'Samiam',
 'Captain Beefheart and His Magic Band',
 'Ten Foot Pole',
 'Further Seems Forever',
 'Skinny Puppy',
 'Exhorder',
 'Satyricon',
 'Savage Garden',
 'John Lee Hooker',
 'Midlake',
 'Brian Eno and David Byrne',
 'Sponge',
 'Count Five',
 'Defi

In [110]:
# I have to import this df for the function to properly work
df = pd.read_csv('Datasets/df_blend_ratings.csv')
artists = df['artist'].unique()
len(artists)

4249

In [11]:
artists[-1]

'As Living Arrows'

In [12]:
df_artists_origins.index[-1]

3423

In [13]:
df_artists_origins.tail()

,artist,origin
3419,Cinderella,"Philadelphia, Pennsylvania, U.S."
3420,Death From Above 1979,"Toronto, Ontario, Canada"
3421,Chicago,"Chicago, Illinois, United States"
3422,The Nation of Ulysses,"Washington, D.C."
3423,Strung Out,"Simi Valley, California, U.S."


In [164]:
np.where(artists=='July Talk')

(array([2772], dtype=int64),)

In [163]:
artists[2772]

'July Talk'

## **Code to execute the functions from ``geopy_functions.py``**

In [406]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists_us_to_do = []

for artist in artists_us:
    if artist not in df_artists_origins['artist'].values:
        artists_us_to_do.append(artist)

len(artists_us_to_do)

156

In [374]:
# create the df with the origins scraped from Wikipedia

df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
start_index = 0
final_index = start_index+10

df_artists_origins = get_origins_wikipedia(df, start_index, final_index)

Bingo! df_coordinates.csv found 

1/1 - Threshold_(band): Surrey, England
2/2 - Shuggie_Otis (individual): Los Angeles, California, U.S.
2/3 - Morgue: error
3/4 - Divine_Styler: Brooklyn, New York, U.S.
4/5 - Teenage_Mutant_Ninja_Turtles: al workTeenage Mutant Ninja Turtles(1984)OwnersMirage Studios (1984–2009) Nickelodeon (2009–present)Years1984–presentPrint publicationsComicsList of comicsComic strip(s)See list of comicsFilms and televisionFilm(s)List of filmsTelevision seriesList of television seriesAnimated seriesSee list of television seriesTelevision film(s)See list of filmsDirect-to-videoSee list of filmsGamesRole-playingTeenage Mutant Ninja Turtles & Other StrangenessVideo game(s)List of video gamesMiscellaneousToy(s)Action figuresLego Teenage Mutant Ninja TurtlesCharactersList of characters
5/6 - 45_Grave: Los Angeles, California, U.S.
6/7 - Spinal_Tap_(band): als, the New 
7/8 - Only_Living_Witness: Boston, Massachusetts, U.S.
8/9 - Goatlord_(band): Las Vegas, Nevada, United 

In [375]:
# create a subset of the new artists I just got, and tell me if there are nulls
df_new_artists = get_new_artists(df_artists_origins)

# show the first new artists, if they were duplicates they have been dropped
df_new_artists.head()

2 nulls (20.0 %)


,artist,origin
0,Threshold,"Surrey, England"
1,Shuggie Otis,"Los Angeles, California, U.S."
2,Morgue,NaN
3,Divine Styler,"Brooklyn, New York, U.S."
4,Teenage Mutant Ninja Turtles,al workTeenage Mutant Ninja Turtles(1984)Owner...


In [376]:
df_new_artists.shape

(10, 2)

### If there are null or weird values

In [377]:
nulls = df_new_artists[df_new_artists['origin'].isna()]
nulls.shape

(2, 2)

In [378]:
nulls

,artist,origin
2,Morgue,NaN
9,Resurrection,NaN


In [327]:
df_new_artists[df_new_artists['origin'].isna()]

,artist,origin


**``np.where`` to replace the values for the real origins**

In [379]:
for artist in nulls['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "England", df_new_artists["origin"])')

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Morgue", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Resurrection", "England", df_new_artists["origin"])


In [392]:
# looking in the internet for the real origins of these artists

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Morgue", "Plainfield, IL, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Resurrection", "Tampa, FL, United States", df_new_artists["origin"])

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Spinal Tap", "Los Angeles, CA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])

In [391]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Spinal Tap	".strip()].head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
6063,1992,Spinal Tap,Break Like the Wind,2.88,33,49.75,14,['Heavy Metal'],US,"A hard rock parody band, made up for the mocku..."


In [318]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[5317]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Death / Thrash Metal band from Winthrop Harbor, Illinois, United States.  Active years : 1985-1991, 1994-1996,
2000-2001, 2020-present.


In [ ]:
df_ratings_20[df_ratings_20['artist']=='Embrace']

#### Changing ``df_ratings_20`` in case needed

In [190]:
df_ratings_20.loc[8662, 'artist'] = 'Embrace (US)'
df_ratings_20.loc[4493, 'artist'] = 'Embrace (UK)'
df_ratings_20.loc[4494, 'artist'] = 'Embrace (UK)'

In [ ]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [214]:
df_new_artists['origin'].isna().sum()

0

#### **Drop artists that are not from the UK or the US**

In [388]:
artists_to_remove = ['Teenage Mutant Ninja Turtles']

for artist in artists_to_remove:
    try:
        artists_us_to_do.remove(artist)
        artists_us.remove(artist)
        print(f'{artist} removed')
    except:
        print('artist not found')

Teenage Mutant Ninja Turtles removed


In [387]:
# if they are not brittish or american, I can drop them
print(df_new_artists.shape)
df_new_artists.drop([4], axis=0, inplace=True)
print(df_new_artists.shape)

(10, 2)
(9, 2)


#### **Check long and short origins, probably wrong**

In [393]:
start_index = 4

end_index = start_index+5
df_new_artists.loc[start_index:end_index]

,artist,origin
5,45 Grave,"Los Angeles, California, U.S."
6,Spinal Tap,"Los Angeles, CA, United States"
7,Only Living Witness,"Boston, Massachusetts, U.S."
8,Goatlord,"Las Vegas, Nevada, United States"
9,Resurrection,"Tampa, FL, United States"


In [394]:
# print abnormally long origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) > 45:
        print(index, row['origin'])

In [395]:
# print abnormally short origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) < 10:
        print(index, row['origin'])

### **Changing easy values: individuals that didn't get the right origin in Wikipedia**

In [345]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains('Genres')]

,artist,origin


In [346]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains(' and ')]

,artist,origin


In [347]:
# bands from Cumbria, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Cumbria')]

,artist,origin


In [348]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace('Cumbria', 'Cumberland'))

In [349]:
# bands from Middlesex, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Middlesex')]

,artist,origin


In [350]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Middlesex', ''))

In [351]:
# bands from Yorkshire, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('West Riding of Yorkshire')]

,artist,origin


In [352]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', West Riding of Yorkshire', ''))

In [353]:
# bands from Merseyside, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Merseyside')]

,artist,origin


In [354]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Merseyside', ''))

In [355]:
# bands from United Kingdom, wrong origin, poor level of detail
df_new_artists[df_new_artists['origin']==('United Kingdom')]

,artist,origin


### **``np.where`` to replace the values for the real origins**

In [361]:
# looking on the internet for the real origins of these artists

df_new_artists['origin'] = np.where(df_new_artists['artist']=="Hammerhead", "Moorhead, Minnesota, United States", df_new_artists['origin'])


**Try a single origin in Geopy**

In [359]:
# try to get the right origin of an origin that crashed
geolocator = Nominatim(user_agent="music_analysis")

origin = "Long Island, New York, United States"

origin_clean = re.sub(r'\[\d+\]', '', origin).replace('.', '')
location = geolocator.geocode(origin_clean)
print(f"{location.address}")
print(location.latitude, location.longitude)

Long Island, New York, United States
40.85149705 -73.09942966988302


In [ ]:
start_index = 0

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
32,Throwing Muses,"Newport, Rhode Island, United States"
33,Razor,"Guelph, Ontario, Canada"
34,The Gories,"Detroit, Michigan, United States"
35,Hammerhead,"Moorhead, Minnesota, United States"
37,Warship,"Long Island, New York, United States"


**Try all origins in Geopy**

In [396]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 0
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

0 - Surrey, England, United Kingdom
1 - Los Angeles, Los Angeles County, California, United States
2 - Plainfield, Plainfield Township, Will County, Illinois, United States
3 - Brooklyn, Kings County, City of New York, New York, United States
4 - Los Angeles, Los Angeles County, California, United States
5 - Los Angeles, Los Angeles County, California, United States
6 - Boston, Suffolk County, Massachusetts, United States
7 - Las Vegas, Clark County, Nevada, United States
8 - Tampa, Hillsborough County, Florida, United States


## **Export to .csv**

### GeoPy wrong locations

**In case GeoPy fails due to a wrong location, I have to delete the new locations, export again, change the location and run GeoPy again**

In [ ]:
#     # import the df with the artists' origins already scraped
# df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
# df_artists_origins_scraped = df_artists_origins_scraped[0:-20]
# df_artists_origins_scraped.to_csv('Datasets/df_artists_origins.csv', index=False)

In [397]:
df_new_artists.shape

(9, 2)

In [398]:
# in case I created by mistake already 'origin_clean' and I want to drop it
# df_new_artists = df_new_artists[['artist', 'origin']]
# df_new_artists

### **Export to .csv**

In [399]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3620, 2)


## **GeoPy**

In [400]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/8 - Surrey, England, United Kingdom
2/8 - Los Angeles, Los Angeles County, California, United States
3/8 - Plainfield, Plainfield Township, Will County, Illinois, United States
4/8 - Brooklyn, Kings County, City of New York, New York, United States
5/8 - Los Angeles, Los Angeles County, California, United States
6/8 - Boston, Suffolk County, Massachusetts, United States
7/8 - Las Vegas, Clark County, Nevada, United States
8/8 - Tampa, Hillsborough County, Florida, United States


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Surrey,"Surrey, England","Surrey, England",51.271532,-0.341452,"Surrey, England, United Kingdom"
1,United States,Boston,"Boston, Massachusetts, U.S.","Boston, Massachusetts, US",42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
2,United States,Brooklyn,"Brooklyn, New York, U.S.","Brooklyn, New York, US",40.652601,-73.949721,"Brooklyn, Kings County, City of New York, New ..."
3,United States,Las Vegas,"Las Vegas, Nevada, United States","Las Vegas, Nevada, United States",36.167256,-115.148516,"Las Vegas, Clark County, Nevada, United States"
4,United States,Los Angeles,"Los Angeles, California, U.S.","Los Angeles, California, US",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
5,United States,Los Angeles,"Los Angeles, CA, United States","Los Angeles, CA, United States",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Plainfield,"Plainfield, IL, United States","Plainfield, IL, United States",41.608671,-88.205434,"Plainfield, Plainfield Township, Will County, ..."
7,United States,Tampa,"Tampa, FL, United States","Tampa, FL, United States",27.947760,-82.458444,"Tampa, Hillsborough County, Florida, United St..."


In [401]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1661, 7)

Found 7 duplicates:
             city         country
719        Surrey  United Kingdom
886        Boston   United States
903      Brooklyn   United States
1149    Las Vegas   United States
1191  Los Angeles   United States
1192  Los Angeles   United States
1542        Tampa   United States

Resulting dataset: (1662, 7)
Merged artists with coordinates! Found 1 new locations
df_coordinates_concat exported to .csv


In [402]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3615,45 Grave,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
3616,Spinal Tap,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
3617,Only Living Witness,United States,Boston,42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
3618,Goatlord,United States,Las Vegas,36.167256,-115.148516,"Las Vegas, Clark County, Nevada, United States"
3619,Resurrection,United States,Tampa,27.947760,-82.458444,"Tampa, Hillsborough County, Florida, United St..."


## **Statistics**

In [403]:
df_artists_origins_coordinates_concat = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')
df_artists_origins_coordinates_concat.shape

(3620, 6)

In [404]:
brittish_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United Kingdom']
american_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']

In [405]:
order = df_artists_origins_coordinates_concat['country'].value_counts().index
print(df_artists_origins_coordinates_concat['country'].value_counts().head())

# plt.figure(figsize=(8,6))
# sns.countplot(df_artists_origins_coordinates_concat['country'], order=order)

country
United States     1860
United Kingdom    1176
Canada             115
Sverige             89
Australia           77
Name: count, dtype: int64


In [177]:
print(f"{brittish_cities.shape[0]} Brittish artists")
order = brittish_cities['city'].value_counts().index
print(f"{brittish_cities['city'].nunique()} Brittish cities\n")
print(brittish_cities['city'].value_counts().head(15))

# plt.figure(figsize=(9,45))
# sns.countplot(brittish_cities['city'], order=order)

1169 Brittish artists
379 Brittish cities

city
London                 309
Glasgow                 37
Birmingham              34
Manchester              29
Leeds                   25
Liverpool               24
Brighton                18
Bristol                 17
Sheffield               15
Edinburgh               13
Cardiff                 12
Nottingham              12
Reading                 10
Essex                    9
Newcastle upon Tyne      9
Name: count, dtype: int64


In [138]:
print(f"{american_cities.shape[0]} American artists")
order = american_cities['city'].value_counts().index
print(f"{american_cities['city'].nunique()} American cities\n")
print(american_cities['city'].value_counts().head(15))

# plt.figure(figsize=(5,55))
# sns.countplot(df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']['city'], order=order)

1713 American artists
519 American cities

city
Los Angeles      195
New York City    124
San Francisco     57
Chicago           56
Seattle           44
Brooklyn          37
Boston            30
Philadelphia      27
Portland          25
San Diego         21
Phoenix           18
Austin            18
Washington        18
New York          16
Detroit           16
Name: count, dtype: int64
